In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import re

In [2]:
url = 'https://ev-database.org/#group=vehicle-group&rs-pr=10000_100000&rs-er=0_1000&rs-ld=0_1000&rs-ac=2_23&rs-dcfc=0_400&rs-ub=10_200&rs-tw=0_2500&rs-ef=100_350&rs-sa=-1_5&rs-w=1000_3500&rs-c=0_5000&rs-y=2010_2030&s=1&p=0-10'
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
    }

# Loading the webpage using requsts library
response = requests.get(url, headers= headers)
print(response)

html_code=response.text

print(html_code[:2000])

Title = []


soup = BeautifulSoup(html_code, "html.parser")

Name = soup.find_all('div', attrs={'class':'item-data'})
for h in Name:
    Title.append(h.text.strip())

<Response [200]>

<!doctype html>

<html lang="en" data-locale="en-IE" data-region="si">
	<!-- head -->
	<head>		
	<meta charset="utf-8" />
			<link rel="canonical" href="https://ev-database.org/">	<link rel="alternate" hreflang="en-gb" href="https://ev-database.org/uk/">	<link rel="alternate" hreflang="de-de" href="https://ev-database.org/de/">	<link rel="alternate" hreflang="nl-nl" href="https://ev-database.org/nl/">	<link rel="alternate" hreflang="x-default" href="https://ev-database.org/">	<meta name="viewport" content="width=device-width, initial-scale=1" />
	<title>All electric vehicles in Europe - EV Database</title>
		<meta name="description" content="A complete overview of all electric vehicles in Europe. Search and compare by range, make, model and price.">

	<meta property="og:title" content="EV Database" />
	<meta property="og:description" content="A complete overview of all electric vehicles in Europe. Search and compare by range, make, model and price." />
	<meta property

In [3]:
Title

['BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive \n\n\t\t\t\t\t\tAvailable to order since September 2025\t\t\t\t\t\n\n\n\n\nD\n\n5\nHeatpump\n\n \n\nTowbar possible\n\n\n\n-1V2L\nV2XLH\nV2XHG\nV2XG \n\n\n\nRange*\n610 km\n\n\nEfficiency*\n178 Wh/km\n\n\nWeight\n2360 kg\n2360\n\n\n0-100\n4.9 sec\n4.9\n\n\n1-Stop Range\n742 km\n742\n\n\nBattery\n108.7 kWh\n108.7\n\n\nFastcharge\n225 kW\n225\n\n\nTowing\n2000 kg\n2000\n\n\nCargo Vol.\n578 L\n578\n\n\nPrice/range*\n€113 /km\n113\n\n\n\n\n\n€68,900\n\n\n\n€70,501\n\n\n\n£58,755\n\n\n68989\n68989',
 'MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh \n\n\t\t\t\t\t\tAvailable to order since October 2022\t\t\t\t\t\n\n\n\n\nC\n\n5\nHeatpump\n\n \n\nTowbar possible\n\n\n\n5\n5V2L\nV2XLHG \n\n\n\nRange\n360 km\n\n\nEfficiency\n171 Wh/km\n\n\nWeight\n1726 kg\n1726\n\n\n0-100\n7.9 sec\n7.9\n\n\n1-Stop Range\n405 km\n405\n\n\nBattery\n61.7 kWh\n61.7\n\n\nFastcharge\n115 kW\n115\n\n\nTowing\n500 kg\n500\n\n\nCargo Vol.\n363 L\n363\n\n\nPrice/range\n€102 /km\n1

In [4]:
import re
import pandas as pd

# 👇 Replace this with your variable name
data_list = Title    # your variable that contains all 1142 raw entries

cleaned_data = []

for text in data_list:
    car = {}
    
    # --- Core Details ---
    car['Model'] = re.search(r'^[A-Za-z0-9\s\-\+()]+', text).group().strip() if re.search(r'^[A-Za-z0-9\s\-\+()]+', text) else None
    car['Status'] = re.search(r'(Available to order since [A-Za-z]+\s*\d{4}|Discontinued\s*\([^)]+\))', text)
    car['Status'] = car['Status'].group(1) if car['Status'] else None

    # --- Numeric Specs ---
    def get_float(pattern, unit):
        m = re.search(pattern, text)
        return float(m.group(1).replace(',', '')) if m else None

    car['Range_km']              = get_float(r'Range\*?\s*([\d,.]+)\s*km', 'km')
    car['Efficiency_Wh_per_km']  = get_float(r'Efficiency\*?\s*([\d,.]+)\s*Wh/km', 'Wh/km')
    car['Weight_kg']             = get_float(r'Weight\s*([\d,.]+)\s*kg', 'kg')
    car['Zero_to_Hundred_sec']   = get_float(r'0-100\s*([\d,.]+)\s*sec', 'sec')
    car['One_Stop_Range_km']     = get_float(r'1-Stop Range\s*([\d,.]+)\s*km', 'km')
    car['Battery_kWh']           = get_float(r'Battery\*?\s*([\d,.]+)\s*kWh', 'kWh')
    car['Fastcharge_kW']         = get_float(r'Fastcharge\*?\s*([\d,.]+)\s*kW', 'kW')
    car['Towing_kg']             = get_float(r'Towing\s*([\d,.]+)\s*kg', 'kg')
    car['Cargo_L']               = get_float(r'Cargo Vol.\s*([\d,.]+)\s*L', 'L')
    car['Price_per_km_EUR']      = get_float(r'€([\d,.]+)\s*/km', '€/km')

    # --- Prices ---
    euro_prices = re.findall(r'€([\d,.]+)', text)
    car['Price_EUR'] = float(euro_prices[0].replace(',', '')) if euro_prices else None
    gbp_price = re.search(r'£([\d,.]+)', text)
    car['Price_GBP'] = float(gbp_price.group(1).replace(',', '')) if gbp_price else None

    cleaned_data.append(car)

# --- Create DataFrame ---
df_ev = pd.DataFrame(cleaned_data)

# --- Clean up ---
df_ev.drop_duplicates(subset=["Model"], inplace=True)
df_ev.reset_index(drop=True, inplace=True)

print("✅ Extracted and cleaned successfully!")
print("Total rows:", len(df_ev))
df_ev

# Optional: Save for EDA
# df_ev.to_csv("electric_cars_cleaned.csv", index=False)


✅ Extracted and cleaned successfully!
Total rows: 980


,Model,Status,Range_km,Efficiency_Wh_per_km,Weight_kg,Zero_to_Hundred_sec,One_Stop_Range_km,Battery_kWh,Fastcharge_kW,Towing_kg,Cargo_L,Price_per_km_EUR,Price_EUR,Price_GBP
0,BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive \n\n...,Available to order since September 2025,610.0,178.0,2360.0,4.9,742.0,108.7,225.0,2000.0,578.0,113.0,113.0,58755.0
1,MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh \...,Available to order since October 2022,360.0,171.0,1726.0,7.9,405.0,61.7,115.0,500.0,363.0,102.0,102.0,29495.0
2,Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+ \...,Available to order since May 2025,585.0,145.0,2055.0,6.7,763.0,85.0,235.0,1500.0,506.0,95.0,95.0,45615.0
3,BMW iX xDrive40BMW iX xDrive40 \n\n\t\t\t\t\t\...,Discontinued (October 2021 - January 2025),360.0,197.0,2440.0,6.1,388.0,71.0,102.0,2500.0,500.0,225.0,225.0,69905.0
4,Fiat 500e Hatchback 42 kWhFiat 500e Hatchback ...,Available to order since November 2020,235.0,159.0,1365.0,9.0,261.0,37.3,67.0,0.0,185.0,143.0,143.0,27995.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,Lucid Air Touring (MY25)Lucid Air Touring \n\n...,Discontinued (September 2024 - August 2025),605.0,159.0,2337.0,3.6,689.0,96.0,160.0,0.0,910.0,192.0,192.0,NaN
976,Opel Vivaro-e Combi M 50 kWhOpel Vivaro-e Comb...,Discontinued (October 2020 - January 2024),185.0,250.0,1969.0,12.1,198.0,46.3,78.0,1000.0,507.0,255.0,255.0,36995.0
977,Peugeot e-Expert Combi Long 50 kWhPeugeot e-Ex...,Discontinued (February 2021 - January 2024),185.0,250.0,1989.0,13.1,198.0,46.3,78.0,1000.0,989.0,278.0,278.0,NaN
978,Peugeot e-Traveller Compact 50 kWhPeugeot e-Tr...,Discontinued (January 2021 - January 2022),185.0,250.0,1932.0,12.1,191.0,46.3,78.0,1000.0,NaN,275.0,275.0,NaN


In [5]:
# Example DataFrame name
# df_ev = your existing DataFrame

# 👇 Just edit this list with the column names you want to remove
columns_to_drop = [
    "Price_GBP", 
    "Price_per_km_EUR", 
    "One_Stop_Range_km",
    "Status",
    "Towing_kg"
]

# Drop columns safely
df_ev.drop(columns=columns_to_drop, inplace=True, errors='ignore')

print("✅ Columns dropped successfully!")
print("Remaining columns:")
print(df_ev.columns.tolist())

# Optional: To verify the new structure
df_ev.head(3)


✅ Columns dropped successfully!
Remaining columns:
['Model', 'Range_km', 'Efficiency_Wh_per_km', 'Weight_kg', 'Zero_to_Hundred_sec', 'Battery_kWh', 'Fastcharge_kW', 'Cargo_L', 'Price_EUR']


,Model,Range_km,Efficiency_Wh_per_km,Weight_kg,Zero_to_Hundred_sec,Battery_kWh,Fastcharge_kW,Cargo_L,Price_EUR
0,BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive \n\n...,610.0,178.0,2360.0,4.9,108.7,225.0,578.0,113.0
1,MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh \...,360.0,171.0,1726.0,7.9,61.7,115.0,363.0,102.0
2,Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+ \...,585.0,145.0,2055.0,6.7,85.0,235.0,506.0,95.0


In [6]:
# Conversion rate (you can adjust to current rate)
eur_to_inr = 93.5   # 1 Euro ≈ ₹93.5

# Create new INR price column
df_ev["Price_INR"] = df_ev["Price_EUR"] * eur_to_inr

print("✅ Price successfully converted to INR!")
print(df_ev[["Model", "Price_EUR", "Price_INR"]].head())


✅ Price successfully converted to INR!
                                               Model  Price_EUR  Price_INR
0  BMW iX3 50 xDrive (MY26)BMW iX3 50 xDrive \n\n...      113.0    10565.5
1  MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh \...      102.0     9537.0
2  Mercedes-Benz CLA 250+Mercedes-Benz CLA 250+ \...       95.0     8882.5
3  BMW iX xDrive40BMW iX xDrive40 \n\n\t\t\t\t\t\...      225.0    21037.5
4  Fiat 500e Hatchback 42 kWhFiat 500e Hatchback ...      143.0    13370.5


In [7]:
# Drop rows that have any missing (NaN) values
df_ev.dropna(inplace=True)

print("✅ All rows with missing values dropped successfully!")
print("Remaining rows:", len(df_ev))
print("Remaining columns:", df_ev.columns.tolist())


✅ All rows with missing values dropped successfully!
Remaining rows: 924
Remaining columns: ['Model', 'Range_km', 'Efficiency_Wh_per_km', 'Weight_kg', 'Zero_to_Hundred_sec', 'Battery_kWh', 'Fastcharge_kW', 'Cargo_L', 'Price_EUR', 'Price_INR']


In [8]:
import re

# Clean and extract brand + model properly
df_ev["Model"] = df_ev["Model"].apply(lambda x: re.findall(r'^[A-Za-z0-9\s\-\.]+', str(x).strip())[0] if pd.notnull(x) else x)

# Optional cleanup for extra spaces and newlines
df_ev["Model"] = df_ev["Model"].str.replace(r'\s+', ' ', regex=True).str.strip()

print("✅ Model names cleaned successfully!")
print(df_ev["Model"].head(10))


✅ Model names cleaned successfully!
0                                    BMW iX3 50 xDrive
1    MG MG4 Electric 64 kWhMG MG4 Electric 64 kWh A...
2                                Mercedes-Benz CLA 250
3          BMW iX xDrive40BMW iX xDrive40 Discontinued
4    Fiat 500e Hatchback 42 kWhFiat 500e Hatchback ...
5    CUPRA Born 150 kW - 58 kWhCUPRA Born 150 kW - ...
6                         Tesla Model 3 Long Range RWD
7                         Tesla Model Y Long Range AWD
8    CUPRA Born 170 kW - 77 kWhCUPRA Born 170 kW - ...
9    Kia EV3 Long RangeKia EV3 Long Range Available...
Name: Model, dtype: object


In [9]:
import re

def clean_model_final(text):
    if pd.isna(text):
        return None
    text = str(text)

    # 1️⃣ Remove newlines, tabs, multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # 2️⃣ Remove repeated model parts (like BMW iX3 ... BMW iX3 ...)
    text = re.sub(r'^(.*?)\1', r'\1', text)

    # 3️⃣ Remove unwanted suffixes like "Available to order..." or "Discontinued"
    text = re.sub(r'(Available to order.*|Discontinued.*|since\s+\w+\s+\d{4}|MY\d{2,4}|Heatpump.*)$', '', text, flags=re.IGNORECASE)

    # 4️⃣ Extract clean brand + model name only (letters, numbers, kWh, etc.)
    match = re.search(r'^[A-Za-z0-9\-\s\+\.]+(?:kWh|hp|RWD|AWD|4MATIC|Long Range|Single Motor|Premium|Electric|Coupe|Hatchback|Sedan|SUV)?', text)
    if match:
        clean_name = match.group(0).strip()
    else:
        clean_name = text.strip()

    # 5️⃣ Final cleanup (remove double spaces or trailing numbers)
    clean_name = re.sub(r'\s{2,}', ' ', clean_name).strip()

    return clean_name

# Apply to your dataframe
df_ev["Model_Cleaned"] = df_ev["Model_Cleaned"].apply(clean_model_final)

print("✅ All model names cleaned successfully!")
print(df_ev["Model_Cleaned"].head(20))


KeyError: 'Model_Cleaned'

In [ ]:
df_ev

In [ ]:
import pandas as pd

# ✅ STEP 1: Drop duplicates and clean up (safety)
df_ev = df_ev.drop_duplicates(subset=["Model_Cleaned"]).reset_index(drop=True)

# ✅ STEP 2: Handle missing numeric columns for safe sampling
numeric_cols = ["Range_km", "Price_EUR", "Efficiency_Wh_per_km"]
for col in numeric_cols:
    if col in df_ev.columns:
        df_ev[col] = pd.to_numeric(df_ev[col], errors="coerce")
        df_ev[col].fillna(df_ev[col].median(), inplace=True)

# ✅ STEP 3: Create category for diversity (low/mid/high range)
if "Range_km" in df_ev.columns:
    df_ev["Range_Category"] = pd.qcut(df_ev["Range_km"], q=4, labels=["Low", "Medium", "High", "Ultra"])

# ✅ STEP 4: Stratified sampling → ensures balanced variety
df_ev_400 = (
    df_ev.groupby("Range_Category", group_keys=False)
    .apply(lambda x: x.sample(min(len(x), 100), random_state=42))
    .reset_index(drop=True)
)

# ✅ STEP 5: If less than 400 (due to small categories), top up with random
if len(df_ev_400) < 400:
    needed = 400 - len(df_ev_400)
    extra = df_ev.drop(df_ev_400.index).sample(n=needed, random_state=42)
    df_ev_400 = pd.concat([df_ev_400, extra], ignore_index=True)

# ✅ STEP 6: Reset index and show result
df_ev_400 = df_ev_400.reset_index(drop=True)

print(f"✅ Extracted {len(df_ev_400)} diverse electric cars successfully!")
print(df_ev_400[["Model_Cleaned", "Range_km", "Price_EUR", "Efficiency_Wh_per_km"]].head(10))



In [ ]:
df_ev_400

In [10]:
df_ev_400.info()

NameError: name 'df_ev_400' is not defined

In [ ]:
# 💾 Save your final 400-row EV dataset to CSV
df_ev_400.to_csv("eda_ev400.csv", index=False, encoding="utf-8")

print("✅ Successfully saved your 400-row dataset!")
print("📂 File name: eda_ev400.csv")
print(f"🧾 Total rows saved: {len(df_ev_400)}")


# Ev_data cleaning...............................................................................

In [ ]:
df_ev_400
